# Классификация комментариев

Интернет-магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Требуется обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Требуется построить модель со значением метрики качества *F1* не меньше 0.75. 

<h1>Содержание<span class="tocSkip"></span></h1>
</li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

## Подготовка

In [1]:
import pandas as pd
import numpy as np
import torch
import transformers
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score, train_test_split 
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import GridSearchCV
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import SnowballStemmer

In [2]:
data = pd.read_csv('datasets/toxic_comments.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Оценим баланс классов в исходном датасете.

In [6]:
data['toxic'].value_counts()[1]/data['toxic'].value_counts()[0]

0.1131876717871444

Баланс классов меньше 0.5, поэтому при разделении на выборки будем использовать стратификацию.

Инициализируем токенизатор:

In [7]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

Проверим работу:

In [8]:
tokenizer.encode('Hello world', add_special_tokens=True) 

[101, 7592, 2088, 102]

Токенизируем слова в `data['text']`:

In [9]:
tokenized = data['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True))
tokenized

Token indices sequence length is longer than the specified maximum sequence length for this model (631 > 512). Running this sequence through the model will result in indexing errors


0         [101, 7526, 2339, 1996, 10086, 2015, 2081, 210...
1         [101, 1040, 1005, 22091, 2860, 999, 2002, 3503...
2         [101, 4931, 2158, 1010, 1045, 1005, 1049, 2428...
3         [101, 1000, 2062, 1045, 2064, 1005, 1056, 2191...
4         [101, 2017, 1010, 2909, 1010, 2024, 2026, 5394...
                                ...                        
159566    [101, 1000, 1024, 1024, 1024, 1024, 1024, 1998...
159567    [101, 2017, 2323, 2022, 14984, 1997, 4426, 200...
159568    [101, 13183, 6290, 26114, 1010, 2045, 2015, 20...
159569    [101, 1998, 2009, 3504, 2066, 2009, 2001, 2941...
159570    [101, 1000, 1998, 1012, 1012, 1012, 1045, 2428...
Name: text, Length: 159571, dtype: object

Посмотрим, какова максимальная длина строки.

In [10]:
def maxlen(df):
    max_len = 0
    for i in df.values:
        if len(i) > max_len:
            max_len = len(i)
    return max_len

In [11]:
maxlen(tokenized)

4950

Токенов в строке многовато для BERT, а наш датасет слишком велик, и попытка построени embeddings на всём датасете на моём ноутбуке с 16 Гб оперативной памяти ориентировочно занимает 75 часов. Уменьшим датасет, чтобы сэкономить время.

Чтобы модель BERT могла работать, оставим только строки, не превышающие 512 токенов.

In [12]:
tokenized512 = tokenized[tokenized.map(len) <= 512]
tokenized512

0         [101, 7526, 2339, 1996, 10086, 2015, 2081, 210...
1         [101, 1040, 1005, 22091, 2860, 999, 2002, 3503...
2         [101, 4931, 2158, 1010, 1045, 1005, 1049, 2428...
3         [101, 1000, 2062, 1045, 2064, 1005, 1056, 2191...
4         [101, 2017, 1010, 2909, 1010, 2024, 2026, 5394...
                                ...                        
159566    [101, 1000, 1024, 1024, 1024, 1024, 1024, 1998...
159567    [101, 2017, 2323, 2022, 14984, 1997, 4426, 200...
159568    [101, 13183, 6290, 26114, 1010, 2045, 2015, 20...
159569    [101, 1998, 2009, 3504, 2066, 2009, 2001, 2941...
159570    [101, 1000, 1998, 1012, 1012, 1012, 1045, 2428...
Name: text, Length: 156048, dtype: object

Убедимся в том, что максимальная длина строки не превышает 512 токенов.

In [13]:
max_len = maxlen(tokenized512)
max_len

512

Сделаем выборку.

In [14]:
tokenized512 = tokenized512.sample(6000, random_state=4)
tokenized512

79345     [101, 9268, 5310, 5857, 2017, 2079, 2025, 6366...
56195     [101, 1000, 1024, 1024, 1024, 2045, 2003, 2023...
88061     [101, 4512, 2007, 2033, 3849, 2066, 2017, 9471...
117948    [101, 2045, 1005, 1055, 7564, 1997, 6971, 1394...
89217     [101, 9610, 13327, 2846, 2045, 2323, 2022, 103...
                                ...                        
63806     [101, 1000, 3531, 2079, 2025, 3158, 9305, 4697...
92596     [101, 1045, 2903, 2057, 2323, 5787, 11619, 199...
74153     [101, 2748, 1045, 1042, 1008, 23616, 2075, 207...
103249       [101, 1012, 2009, 2003, 2525, 1999, 2486, 102]
68436     [101, 1000, 1024, 2004, 2000, 2115, 3160, 1024...
Name: text, Length: 6000, dtype: object

Применим метод padding (англ. «отступ»), чтобы после токенизации длины исходных текстов в корпусе были равными. Только при таком условии будет работать модель BERT. Пусть стандартной длиной вектора n будет длина наибольшего во всём датасете вектора. Остальные векторы дополним нулями:

In [15]:
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized512.values])
padded

array([[ 101, 9268, 5310, ...,    0,    0,    0],
       [ 101, 1000, 1024, ...,    0,    0,    0],
       [ 101, 4512, 2007, ...,    0,    0,    0],
       ...,
       [ 101, 2748, 1045, ...,    0,    0,    0],
       [ 101, 1012, 2009, ...,    0,    0,    0],
       [ 101, 1000, 1024, ...,    0,    0,    0]])

Теперь поясним модели, что нули не несут значимой информации. Это нужно для компоненты модели, которая называется «внимание» (англ. attention). Отбросим эти токены и «создадим маску» для действительно важных токенов, то есть укажем нулевые и не нулевые значения:

In [16]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

Инициализируем конфигурацию BertConfig.

In [17]:
config = transformers.BertConfig.from_pretrained('bert-base-uncased')

Инициализируем саму модель класса BertModel. Передадим ей файл с предобученной моделью и конфигурацией:

In [18]:
model = transformers.BertModel.from_pretrained('bert-base-uncased', config=config)

In [19]:
#data

In [20]:
#torch.LongTensor(padded[500:1000])

In [21]:
batch_size = 500
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    """
    Преобразуем данные в формат тензоров (англ. tensor) — многомерных векторов 
    в библиотеке torch. Тип данных LongTensor (англ. «длинный тензор») хранит 
    числа в «длинном формате», то есть выделяет на каждое число 64 бита.
    """
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
    # преобразуем маску
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
    """
    Для ускорения вычисления функцией no_grad() (англ. no gradient, «нет градиента») 
    в библиотеке torch укажем, что градиенты не нужны: модель BERT обучать не будем.
    """
    with torch.no_grad():
        #Чтобы получить эмбеддинги для батча, передадим модели данные и маску:
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)
    """
    Из полученного тензора извлечём нужные элементы и добавим в список всех эмбеддингов:
    """
    embeddings.append(batch_embeddings[0][:,0,:].numpy())

Соберём все эмбеддинги в матрицу признаков:

In [22]:
features = np.concatenate(embeddings)
len(features)

6000

In [23]:
target = data.loc[tokenized512.index]['toxic']
len(target)

6000

Мы подготовили признаки и целевой признак.

## Обучение

Разобьём датасет на обучающую и тестовую выборки.

In [25]:
features_train, features_test, target_train, target_test = \
train_test_split(features, target, test_size = 0.2, stratify=target, random_state=12345)

In [28]:
# Функция возвращает предсказания лучшей модели
def best_model(features_train, 
                  target_train,
                  model,
                  param_grid, 
                  cv, 
                  refit):
    
    my_scorer = {'F1': make_scorer(f1_score)
                #,'MAE': make_scorer(mean_absolute_error)
                }
    
    model = model
    grid = GridSearchCV(model, 
                        param_grid, 
                        cv=cv, 
                        refit=refit, 
                        scoring=my_scorer) 
    grid_result = grid.fit(features_train, target_train)
    best_model = grid_result.best_estimator_
    best_score = grid_result.best_score_
    best_params = grid.best_params_
    print(best_params)
    #print(refit, ": {:.2f}".format(best_score), "%")
    print(refit,":", best_score)
    return(best_model, best_score)

Обучим различные модели.

In [57]:
# Сетка параметров для логистической регрессии
PARAM_GRID_LR = {#'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                 'max_iter': [10000],
                 'class_weight': ['balanced'],
                 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
                 ,'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
                }
PARAM_GRID_LR;

In [58]:
%%time
model_LR, F1_LR = best_model(features_train, 
                  target_train,
                  LogisticRegression(),
                  PARAM_GRID_LR, 
                  5, 
                  'F1')

C:\Users\shepelev.ao\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\shepelev.ao\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:204: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
C:\Users\shepelev.ao\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\shepelev.ao\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:204: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
C:\Users\shepelev.ao\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\shepelev.ao\Anaconda3\lib\site-packages\

C:\Users\shepelev.ao\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\shepelev.ao\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:204: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
C:\Users\shepelev.ao\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\shepelev.ao\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:204: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
C:\Users\shepelev.ao\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\shepelev.ao\Anaconda3\lib\site-packages\

{'C': 1, 'class_weight': 'balanced', 'max_iter': 10000, 'solver': 'liblinear'}
F1 : 0.6221613123277205
Wall time: 1h 4min 22s


{'class_weight': 'balanced', 'max_iter': 10000, 'solver': 'liblinear'}
F1 : 0.6221613123277205
Wall time: 2min 23s

In [33]:
# Сетка параметров для DecisionTreeClassifier()
PARAM_GRID_DTC = {'criterion': ['gini', 'entropy'],
                  'splitter': ['best', 'random'],
                  'max_depth': range(1,20),
                  'min_samples_leaf': range(1,10),
                  'random_state': [12345]
                 }  
PARAM_GRID_DTC;

In [34]:
%%time
model_DTC, F1_DTC = best_model(features_train, 
                  target_train,
                  DecisionTreeClassifier(),
                  PARAM_GRID_DTC, 
                  5, 
                  'F1')

{'criterion': 'entropy', 'max_depth': 7, 'min_samples_leaf': 9, 'random_state': 12345, 'splitter': 'random'}
F1 : 0.42693844403029546
Wall time: 1h 14min 14s


{'criterion': 'entropy', 'max_depth': 7, 'min_samples_leaf': 9, 'random_state': 12345, 'splitter': 'random'}
F1 : 0.42693844403029546
Wall time: 1h 14min 14s

In [35]:
# Сетка параметров для RandomForestClassifier()
PARAM_GRID_RFC = {'n_estimators': [3,  10,  31, 100], #np.logspace(0, 2, 5, endpoint=True, dtype = 'int')
                  'max_depth': range(1,5),
                  'min_samples_leaf': range(1,5),
                 'random_state': [12345]}  
PARAM_GRID_RFC;

In [36]:
%%time
model_RFC, F1_RFC = best_model(features_train, 
                  target_train,
                  RandomForestClassifier(),
                  PARAM_GRID_RFC, 
                  5, 
                  'F1')

{'max_depth': 4, 'min_samples_leaf': 3, 'n_estimators': 3, 'random_state': 12345}
F1 : 0.18489049978423155
Wall time: 3min 30s


{'max_depth': 4, 'min_samples_leaf': 3, 'n_estimators': 3, 'random_state': 12345}
F1 : 0.18489049978423155
Wall time: 3min 30s

In [37]:
# Сетка параметров для HistGradientBoostingClassifier()
PARAM_GRID_HGBC = {'learning_rate': [0.05, 0.1, 0.2, 0.5, 0.6, 0.7, 0.8, 0.9], #[0.2 ,0.5, 0.7]
                   #'loss': ['auto', 'binary_crossentropy', 'categorical_crossentropy']
                  'max_iter': [   1,   10,  100, 200, 300, 400], #np.logspace(0, 2, 5, endpoint=True, dtype = 'int'),
                  #'max_depth': range(1,5),
                  'random_state': [12345],
                  #'verbose': [1]
                  }  
PARAM_GRID_HGBC;

In [38]:
%%time
model_HGBC, F1_HGBC = best_model(features_train, 
                  target_train,
                  HistGradientBoostingClassifier(),
                  PARAM_GRID_HGBC, 
                  5, 
                  'F1')

{'learning_rate': 0.5, 'max_iter': 300, 'random_state': 12345}
F1 : 0.5590105930737465
Wall time: 48min 46s


{'learning_rate': 0.5, 'max_iter': 300, 'random_state': 12345}
F1 : 0.5590105930737465
Wall time: 48min 46s

In [55]:
# Сетка параметров для CatBoostClassifier()
PARAM_GRID_CBC = {'learning_rate': [0.05, 0.2]#, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 
                  'iterations': [100, 200, 300]#   10,  100, 200], #np.logspace(0, 2, 5, endpoint=True, dtype = 'int'),
                  #'verbose': [10],
                  }  
PARAM_GRID_CBC;

In [56]:
%%time
model_CBC, F1_CBC = best_model(features_train, 
                  target_train,
                  CatBoostClassifier(),
                  PARAM_GRID_CBC, 
                  5, 
                  'F1')

0:	learn: 0.5183328	total: 95.2ms	remaining: 9.43s
1:	learn: 0.4087278	total: 189ms	remaining: 9.26s
2:	learn: 0.3337986	total: 284ms	remaining: 9.19s
3:	learn: 0.2887790	total: 386ms	remaining: 9.26s
4:	learn: 0.2532627	total: 481ms	remaining: 9.13s
5:	learn: 0.2285494	total: 579ms	remaining: 9.07s
6:	learn: 0.2141317	total: 668ms	remaining: 8.87s
7:	learn: 0.2026738	total: 764ms	remaining: 8.78s
8:	learn: 0.1898714	total: 854ms	remaining: 8.64s
9:	learn: 0.1807304	total: 954ms	remaining: 8.59s
10:	learn: 0.1731961	total: 1.05s	remaining: 8.47s
11:	learn: 0.1660728	total: 1.15s	remaining: 8.43s
12:	learn: 0.1603077	total: 1.25s	remaining: 8.34s
13:	learn: 0.1551958	total: 1.35s	remaining: 8.28s
14:	learn: 0.1507237	total: 1.44s	remaining: 8.18s
15:	learn: 0.1457535	total: 1.54s	remaining: 8.09s
16:	learn: 0.1418714	total: 1.63s	remaining: 7.96s
17:	learn: 0.1370607	total: 1.73s	remaining: 7.86s
18:	learn: 0.1333593	total: 1.81s	remaining: 7.74s
19:	learn: 0.1299575	total: 1.91s	remain

62:	learn: 0.0457377	total: 6.18s	remaining: 3.63s
63:	learn: 0.0451512	total: 6.28s	remaining: 3.53s
64:	learn: 0.0442104	total: 6.38s	remaining: 3.44s
65:	learn: 0.0433697	total: 6.48s	remaining: 3.34s
66:	learn: 0.0429784	total: 6.57s	remaining: 3.24s
67:	learn: 0.0420000	total: 6.66s	remaining: 3.13s
68:	learn: 0.0409819	total: 6.76s	remaining: 3.04s
69:	learn: 0.0401225	total: 6.85s	remaining: 2.94s
70:	learn: 0.0394253	total: 6.95s	remaining: 2.84s
71:	learn: 0.0385227	total: 7.04s	remaining: 2.74s
72:	learn: 0.0376331	total: 7.14s	remaining: 2.64s
73:	learn: 0.0368706	total: 7.24s	remaining: 2.54s
74:	learn: 0.0362912	total: 7.34s	remaining: 2.45s
75:	learn: 0.0353170	total: 7.44s	remaining: 2.35s
76:	learn: 0.0348687	total: 7.54s	remaining: 2.25s
77:	learn: 0.0346222	total: 7.63s	remaining: 2.15s
78:	learn: 0.0336221	total: 7.72s	remaining: 2.05s
79:	learn: 0.0329587	total: 7.82s	remaining: 1.96s
80:	learn: 0.0324096	total: 7.92s	remaining: 1.86s
81:	learn: 0.0316506	total: 8.0

24:	learn: 0.1080674	total: 2.75s	remaining: 8.24s
25:	learn: 0.1049514	total: 2.85s	remaining: 8.13s
26:	learn: 0.1032068	total: 2.96s	remaining: 7.99s
27:	learn: 0.1007038	total: 3.06s	remaining: 7.88s
28:	learn: 0.0985040	total: 3.18s	remaining: 7.79s
29:	learn: 0.0956752	total: 3.31s	remaining: 7.72s
30:	learn: 0.0935694	total: 3.43s	remaining: 7.63s
31:	learn: 0.0919623	total: 3.54s	remaining: 7.53s
32:	learn: 0.0905950	total: 3.66s	remaining: 7.43s
33:	learn: 0.0882465	total: 3.77s	remaining: 7.31s
34:	learn: 0.0855737	total: 3.87s	remaining: 7.18s
35:	learn: 0.0824908	total: 3.97s	remaining: 7.06s
36:	learn: 0.0812682	total: 4.07s	remaining: 6.93s
37:	learn: 0.0794211	total: 4.18s	remaining: 6.82s
38:	learn: 0.0771659	total: 4.29s	remaining: 6.72s
39:	learn: 0.0757881	total: 4.4s	remaining: 6.61s
40:	learn: 0.0740499	total: 4.51s	remaining: 6.49s
41:	learn: 0.0722385	total: 4.62s	remaining: 6.38s
42:	learn: 0.0710667	total: 4.73s	remaining: 6.26s
43:	learn: 0.0699612	total: 4.83

86:	learn: 0.0284647	total: 9.42s	remaining: 1.41s
87:	learn: 0.0275672	total: 9.52s	remaining: 1.3s
88:	learn: 0.0272716	total: 9.62s	remaining: 1.19s
89:	learn: 0.0268361	total: 9.73s	remaining: 1.08s
90:	learn: 0.0264495	total: 9.83s	remaining: 972ms
91:	learn: 0.0261358	total: 9.94s	remaining: 864ms
92:	learn: 0.0256877	total: 10s	remaining: 756ms
93:	learn: 0.0253200	total: 10.2s	remaining: 648ms
94:	learn: 0.0250344	total: 10.3s	remaining: 540ms
95:	learn: 0.0243081	total: 10.4s	remaining: 432ms
96:	learn: 0.0240508	total: 10.5s	remaining: 324ms
97:	learn: 0.0234289	total: 10.6s	remaining: 216ms
98:	learn: 0.0231890	total: 10.7s	remaining: 108ms
99:	learn: 0.0223850	total: 10.8s	remaining: 0us
0:	learn: 0.4535525	total: 112ms	remaining: 11.1s
1:	learn: 0.3418698	total: 212ms	remaining: 10.4s
2:	learn: 0.2769111	total: 323ms	remaining: 10.4s
3:	learn: 0.2375101	total: 424ms	remaining: 10.2s
4:	learn: 0.2136130	total: 523ms	remaining: 9.93s
5:	learn: 0.1962600	total: 627ms	remainin

48:	learn: 0.0369613	total: 5.3s	remaining: 5.51s
49:	learn: 0.0361976	total: 5.41s	remaining: 5.41s
50:	learn: 0.0354690	total: 5.51s	remaining: 5.29s
51:	learn: 0.0343027	total: 5.61s	remaining: 5.18s
52:	learn: 0.0332675	total: 5.71s	remaining: 5.07s
53:	learn: 0.0323437	total: 5.82s	remaining: 4.96s
54:	learn: 0.0309793	total: 5.93s	remaining: 4.85s
55:	learn: 0.0297881	total: 6.04s	remaining: 4.75s
56:	learn: 0.0289248	total: 6.16s	remaining: 4.64s
57:	learn: 0.0284061	total: 6.26s	remaining: 4.53s
58:	learn: 0.0275407	total: 6.37s	remaining: 4.43s
59:	learn: 0.0267890	total: 6.47s	remaining: 4.32s
60:	learn: 0.0256850	total: 6.58s	remaining: 4.2s
61:	learn: 0.0248336	total: 6.68s	remaining: 4.1s
62:	learn: 0.0242293	total: 6.8s	remaining: 3.99s
63:	learn: 0.0237434	total: 6.91s	remaining: 3.88s
64:	learn: 0.0230925	total: 7.02s	remaining: 3.78s
65:	learn: 0.0226994	total: 7.12s	remaining: 3.67s
66:	learn: 0.0220784	total: 7.23s	remaining: 3.56s
67:	learn: 0.0217693	total: 7.33s	r

...

121:	learn: 0.0013822	total: 13.1s	remaining: 8.35s
122:	learn: 0.0013045	total: 13.2s	remaining: 8.24s
123:	learn: 0.0012634	total: 13.3s	remaining: 8.13s
124:	learn: 0.0012433	total: 13.4s	remaining: 8.03s
125:	learn: 0.0012083	total: 13.5s	remaining: 7.93s
126:	learn: 0.0012081	total: 13.6s	remaining: 7.82s
127:	learn: 0.0012080	total: 13.7s	remaining: 7.71s
128:	learn: 0.0011975	total: 13.8s	remaining: 7.6s
129:	learn: 0.0011760	total: 13.9s	remaining: 7.49s
130:	learn: 0.0011446	total: 14s	remaining: 7.38s
131:	learn: 0.0011284	total: 14.1s	remaining: 7.28s
132:	learn: 0.0011084	total: 14.2s	remaining: 7.17s
133:	learn: 0.0010855	total: 14.3s	remaining: 7.06s
134:	learn: 0.0010855	total: 14.4s	remaining: 6.96s
135:	learn: 0.0010855	total: 14.6s	remaining: 6.85s
136:	learn: 0.0010854	total: 14.7s	remaining: 6.74s
137:	learn: 0.0010692	total: 14.8s	remaining: 6.63s
138:	learn: 0.0010692	total: 14.9s	remaining: 6.53s
139:	learn: 0.0010599	total: 15s	remaining: 6.42s
140:	learn: 0.001

83:	learn: 0.0026884	total: 9s	remaining: 12.4s
84:	learn: 0.0025891	total: 9.11s	remaining: 12.3s
85:	learn: 0.0025121	total: 9.21s	remaining: 12.2s
86:	learn: 0.0024878	total: 9.31s	remaining: 12.1s
87:	learn: 0.0024182	total: 9.41s	remaining: 12s
88:	learn: 0.0023538	total: 9.52s	remaining: 11.9s
89:	learn: 0.0022819	total: 9.62s	remaining: 11.8s
90:	learn: 0.0022248	total: 9.73s	remaining: 11.7s
91:	learn: 0.0022166	total: 9.84s	remaining: 11.6s
92:	learn: 0.0021821	total: 9.95s	remaining: 11.4s
93:	learn: 0.0021413	total: 10.1s	remaining: 11.3s
94:	learn: 0.0021234	total: 10.2s	remaining: 11.2s
95:	learn: 0.0020891	total: 10.3s	remaining: 11.1s
96:	learn: 0.0020616	total: 10.4s	remaining: 11s
97:	learn: 0.0020255	total: 10.5s	remaining: 10.9s
98:	learn: 0.0019925	total: 10.6s	remaining: 10.8s
99:	learn: 0.0018915	total: 10.7s	remaining: 10.7s
100:	learn: 0.0018496	total: 10.8s	remaining: 10.6s
101:	learn: 0.0018090	total: 10.9s	remaining: 10.5s
102:	learn: 0.0018089	total: 11s	rem

45:	learn: 0.0116696	total: 4.93s	remaining: 16.5s
46:	learn: 0.0112683	total: 5.04s	remaining: 16.4s
47:	learn: 0.0110339	total: 5.15s	remaining: 16.3s
48:	learn: 0.0107042	total: 5.26s	remaining: 16.2s
49:	learn: 0.0102113	total: 5.37s	remaining: 16.1s
50:	learn: 0.0100457	total: 5.47s	remaining: 16s
51:	learn: 0.0099413	total: 5.57s	remaining: 15.9s
52:	learn: 0.0092446	total: 5.67s	remaining: 15.7s
53:	learn: 0.0089621	total: 5.78s	remaining: 15.6s
54:	learn: 0.0087338	total: 5.88s	remaining: 15.5s
55:	learn: 0.0082183	total: 6s	remaining: 15.4s
56:	learn: 0.0078516	total: 6.1s	remaining: 15.3s
57:	learn: 0.0075696	total: 6.21s	remaining: 15.2s
58:	learn: 0.0072441	total: 6.33s	remaining: 15.1s
59:	learn: 0.0069740	total: 6.43s	remaining: 15s
60:	learn: 0.0067424	total: 6.53s	remaining: 14.9s
61:	learn: 0.0060941	total: 6.64s	remaining: 14.8s
62:	learn: 0.0060111	total: 6.74s	remaining: 14.7s
63:	learn: 0.0058973	total: 6.84s	remaining: 14.5s
64:	learn: 0.0056183	total: 6.95s	remai

6:	learn: 0.1141577	total: 778ms	remaining: 21.4s
7:	learn: 0.1058603	total: 884ms	remaining: 21.2s
8:	learn: 0.0969741	total: 986ms	remaining: 20.9s
9:	learn: 0.0894982	total: 1.09s	remaining: 20.7s
10:	learn: 0.0844533	total: 1.19s	remaining: 20.4s
11:	learn: 0.0775351	total: 1.3s	remaining: 20.4s
12:	learn: 0.0679846	total: 1.41s	remaining: 20.3s
13:	learn: 0.0611095	total: 1.52s	remaining: 20.2s
14:	learn: 0.0550419	total: 1.63s	remaining: 20.1s
15:	learn: 0.0505580	total: 1.74s	remaining: 20s
16:	learn: 0.0471836	total: 1.85s	remaining: 19.9s
17:	learn: 0.0418473	total: 1.95s	remaining: 19.7s
18:	learn: 0.0394604	total: 2.05s	remaining: 19.5s
19:	learn: 0.0368128	total: 2.15s	remaining: 19.4s
20:	learn: 0.0345871	total: 2.26s	remaining: 19.2s
21:	learn: 0.0335378	total: 2.37s	remaining: 19.2s
22:	learn: 0.0309443	total: 2.48s	remaining: 19.1s
23:	learn: 0.0287424	total: 2.59s	remaining: 19s
24:	learn: 0.0273259	total: 2.7s	remaining: 18.9s
25:	learn: 0.0251796	total: 2.81s	remaini

167:	learn: 0.0011139	total: 18s	remaining: 3.42s
168:	learn: 0.0011139	total: 18.1s	remaining: 3.32s
169:	learn: 0.0011138	total: 18.2s	remaining: 3.21s
170:	learn: 0.0011138	total: 18.3s	remaining: 3.1s
171:	learn: 0.0011138	total: 18.4s	remaining: 2.99s
172:	learn: 0.0011138	total: 18.5s	remaining: 2.89s
173:	learn: 0.0011138	total: 18.6s	remaining: 2.78s
174:	learn: 0.0011137	total: 18.7s	remaining: 2.67s
175:	learn: 0.0011137	total: 18.8s	remaining: 2.57s
176:	learn: 0.0011137	total: 18.9s	remaining: 2.46s
177:	learn: 0.0011137	total: 19s	remaining: 2.35s
178:	learn: 0.0011137	total: 19.1s	remaining: 2.24s
179:	learn: 0.0011137	total: 19.2s	remaining: 2.14s
180:	learn: 0.0011134	total: 19.3s	remaining: 2.03s
181:	learn: 0.0011134	total: 19.5s	remaining: 1.92s
182:	learn: 0.0011133	total: 19.6s	remaining: 1.82s
183:	learn: 0.0011133	total: 19.7s	remaining: 1.71s
184:	learn: 0.0011133	total: 19.8s	remaining: 1.6s
185:	learn: 0.0011132	total: 19.9s	remaining: 1.5s
186:	learn: 0.00111

{'iterations': 100, 'learning_rate': 0.2}
F1 : 0.563900614864622
Wall time: 22min 9s

Проверим даные на тестовой выборке.

In [61]:
predictions_test = model_LR.predict(features_test)
print("F1 на тестовой выборке:", f1_score(target_test, predictions_test))

F1 на тестовой выборке: 0.6159695817490494


F1 на тестовой выборке: 0.6159695817490494

# Проект для «Викишоп» c TF-IDF

## Подготовка

In [2]:
data = pd.read_csv('datasets/toxic_comments.csv')

In [3]:
data

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


Импортируем стеммер для английского языка.

In [4]:
stemmer = SnowballStemmer('english')

In [5]:
# Функция возвращает преобразованный стеммингом текст
def stemming(text):
    stemmed = []
    for word in text.split():
        stemmed_word = stemmer.stem(word)
        stemmed.append(stemmed_word)
    stemmed_text = " ".join(stemmed)
    return stemmed_text

In [6]:
stemming(data['text'][0])

"explan whi the edit made under my usernam hardcor metallica fan were reverted? they weren't vandalisms, just closur on some gas after i vote at new york doll fac. and pleas don't remov the templat from the talk page sinc i'm retir now.89.205.38.27"

Применим нашу функцию ко всей колонке `text`.

In [7]:
data['text'] = data['text'].apply(stemming)
data

,text,toxic
0,explan whi the edit made under my usernam hard...,0
1,d'aww! he match this background colour i'm see...,0
2,"hey man, i'm realli not tri to edit war. it ju...",0
3,""" more i can't make ani real suggest on improv...",0
4,"you, sir, are my hero. ani chanc you rememb wh...",0
...,...,...
159566,""":::::and for the second time of asking, when ...",0
159567,you should be asham of yourself that is a horr...,0
159568,"spitzer umm, there no actual articl for prosti...",0
159569,and it look like it was actual you who put on ...,0


Разобьём датасет на обучающий и тестовый наборы данных.

In [8]:
train, test = train_test_split(data, test_size=0.2, stratify=data['toxic'],random_state=12345)

In [9]:
train.shape[0]

127656

In [10]:
test.shape[0]

31915

In [11]:
# Функция возвращает матрицы TF-IDF для обучающей и тестовой выборок
def tfidfvectors(train, test, column):
    corpus = train[column].values.astype('U')
    nltk.download('stopwords')
    stopwords = set(nltk_stopwords.words('russian'))
    count_tf_idf = TfidfVectorizer(stop_words=stopwords)
    tf_idf_train = count_tf_idf.fit_transform(corpus)
    tf_idf_test = count_tf_idf.transform(test[column])
    return tf_idf_train, tf_idf_test

In [12]:
features_train, features_test = tfidfvectors(train, test, 'text')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shepelev.ao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
features_train

<127656x159008 sparse matrix of type '<class 'numpy.float64'>'
	with 5479656 stored elements in Compressed Sparse Row format>

In [14]:
features_test

<31915x159008 sparse matrix of type '<class 'numpy.float64'>'
	with 1353941 stored elements in Compressed Sparse Row format>

In [15]:
target_train = train['toxic']
target_test = test['toxic']

## Обучение

In [16]:
# Функция возвращает предсказания лучшей модели
def best_model(features_train, 
                  target_train,
                  model,
                  param_grid, 
                  cv, 
                  refit):
    
    my_scorer = {'F1': make_scorer(f1_score)
                #,'MAE': make_scorer(mean_absolute_error)
                }
    
    model = model
    grid = GridSearchCV(model, 
                        param_grid, 
                        cv=cv, 
                        refit=refit, 
                        scoring=my_scorer) 
    grid_result = grid.fit(features_train, target_train)
    best_model = grid_result.best_estimator_
    best_score = grid_result.best_score_
    best_params = grid.best_params_
    print(best_params)
    #print(refit, ": {:.2f}".format(best_score), "%")
    print(refit,":", best_score)
    return(best_model, best_score)

In [22]:
# Сетка параметров для логистической регрессии
PARAM_GRID_LR = {#'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                 'max_iter': [10000],
                 'class_weight': ['balanced'],
                 'solver': ['lbfgs']# ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
                 #,'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
                }
PARAM_GRID_LR;

In [19]:
%%time
model_LR, F1_LR = best_model(features_train, 
                  target_train,
                  LogisticRegression(),
                  PARAM_GRID_LR, 
                  5, 
                  'F1')

C:\Users\shepelev.ao\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\shepelev.ao\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\shepelev.ao\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1356, in fit
    self.coef_, self.intercept_, n_iter_ = _fit_liblinear(
  File "C:\Users\shepelev.ao\Anaconda3\lib\site-packages\sklearn\svm\_base.py", line 930, in _fit_liblinear
    class_weight_ = compute_class_weight(class_weight, classes=classes_,
  File "C:\Users\shepelev.ao\Anaconda3\lib\site-packages\sklearn\utils\validation.py", line 72, in inner_f
    return f(**kwargs)
  File "C:\Users\shepelev.ao\Anaconda3\lib\site-packages\sklearn\utils\class_weight.p

{'class_weight': 'balanced', 'max_iter': 10000, 'solver': 'lbfgs'}
F1 : 0.7442439227856392
Wall time: 4min 41s


Проверим метрику на тестовой выборке.

In [20]:
predictions_test = model_LR.predict(features_test)
print("F1 на тестовой выборке:", f1_score(target_test, predictions_test))

F1 на тестовой выборке: 0.7599731363331096


Мы получили удовлетворительный результат.

## Выводы

- Наилучший результат на обучающей и тестовой выборке показала модель логистической регрессии.
- Модель BERT для нашей задачи использовать нецелесообразно, т.к. требуется слишком много времени для построения эмбедингов.
- Выборки в 6000 токенов не достаточно для точного предсказания.
- TF-IDF показала лучшие результаты для нашей задачи, и значительно выиграла в скорости.